# Serverless cost simulator for IoT-based traffic

This notebook simulates a synthetic month of requests based on requests generated at configurable periodic intervals.


## 0. Initial setup

### Imports

In [ ]:
from functools import reduce
import awscosts
import bbva_lambda_tools as lambda_tools

### User variables

In [ ]:
image_type = None
# Uncomment next line to produce pngs
# image_type = 'png'

ec2_flavors = ('m3.medium', 'm4.large', 'm4.4xlarge')
req_period_list = [3600]  # , 8* 3600, 24 * 3600]
# [10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000,
# 5000000, 10000000]
num_devices_list = reduce(lambda_tools.devices_func, range(1, 13), [10])
num_devices_list = list(range(100000, 200000000, 100000))
resolution = 60 * 5  # in seconds
interval_duration = 30 * 24 * 3600  # one month in seconds
lambda_memory = 128
lambda_request_duration_ms = 200

### Plotly setup

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

### Common objects

In [ ]:
ec2_instances = [awscosts.EC2(
    flavor,
    MB_per_req=lambda_memory,
    ms_per_req=lambda_request_duration_ms
) for flavor in ec2_flavors]

## 1. Generate and plot costs for IoT-like requests

In [ ]:
for req_period in req_period_list:
    costs = []
    for num_devices in num_devices_list:
        lambda_instance = awscosts.Lambda(
            lambda_memory,
            lambda_request_duration_ms
        )
        cost = lambda_tools.aggregate_costs(
            req_period,
            resolution,
            interval_duration,
            num_devices,
            lambda_instance,
            ec2_instances,
        )

        costs.append(cost)

    figure = lambda_tools.draw_costs_by_num_devices(
        costs,
        ec2_flavors,
        req_period
    )

iplot(figure, image=image_type, image_height=480, image_width=750, filename='serverless.iot.uniform')

## 2. Generate Plots and costs for IoT-like traffic with seasonality

In [ ]:
for req_period in req_period_list:
    costs = []
    for num_devices in num_devices_list:
        lambda_instance = awscosts.Lambda(
            lambda_memory,
            lambda_request_duration_ms
        )

        # Generate Time Series with bumps at the beggining and the end of the time series:
        mean = num_devices // req_period
        devices_time_series = list()
        for i in range(0, (interval_duration // resolution)):
            if i > 0 and i < 1000:
                devices_time_series.append(mean / 2)
            if i >= 1000 and i < 1500:
                devices_time_series.append(mean * 2)
            if i >= 1500 and i < 6500:
                devices_time_series.append(mean / 2)
            if i >= 6500 and i < 8000:
                devices_time_series.append(mean * 2)
            if i >= 8000:
                devices_time_series.append(mean / 2)

        cost = lambda_tools.aggregate_costs(
            req_period,
            resolution,
            interval_duration,
            num_devices,
            lambda_instance,
            ec2_instances,
            devices_time_series,
        )

        costs.append(cost)

    figure = lambda_tools.draw_costs_by_num_devices(
        costs,
        ec2_flavors,
        req_period
    )

iplot(figure, image=image_type, image_height=480, image_width=750, filename='serverless.iot.seasonality')